# Different Ways to Find SQL Server Object Dependencies

https://www.mssqltips.com/sqlservertip/2999/different-ways-to-find-sql-server-object-dependencies/

In [ ]:
--List all access provisioned to a sql user or windows user/group directly -- CHECK

SELECT 
    [UserName] = CASE princ.[type] 
                    WHEN 'S' THEN princ.[name]
                    WHEN 'U' THEN ulogin.[name] COLLATE Latin1_General_CI_AI        END,
    [UserType] = CASE princ.[type]
                    WHEN 'S' THEN 'SQL User'
                    WHEN 'U' THEN 'Windows User'        END,  
    [DatabaseUserName] = princ.[name],       
    [Role] = null,      
    [PermissionType] = perm.[permission_name],       
    [PermissionState] = perm.[state_desc],       
    [ObjectType] = obj.type_desc,--perm.[class_desc],       
    [ObjectName] = OBJECT_NAME(perm.major_id),
    [ColumnName] = col.[name]
FROM sys.database_principals princ  --database user
LEFT JOIN        sys.login_token ulogin on princ.[sid] = ulogin.[sid]     --Login accounts
LEFT JOIN        sys.database_permissions perm ON perm.[grantee_principal_id] = princ.[principal_id] --Permissions
LEFT JOIN        sys.columns col ON col.[object_id] = perm.major_id AND col.[column_id] = perm.[minor_id] --Table columns
LEFT JOIN        sys.objects obj ON perm.[major_id] = obj.[object_id]
WHERE        princ.[type] in ('S','U')

UNION
SELECT  --List all access provisioned to a sql user or windows user/group through a database or application role
    [UserName] = CASE memberprinc.[type] 
                    WHEN 'S' THEN memberprinc.[name]
                    WHEN 'U' THEN ulogin.[name] COLLATE Latin1_General_CI_AI        END,
    [UserType] = CASE memberprinc.[type]
                    WHEN 'S' THEN 'SQL User'
                    WHEN 'U' THEN 'Windows User'        END, 
    [DatabaseUserName] = memberprinc.[name],   
    [Role] = roleprinc.[name],      
    [PermissionType] = perm.[permission_name],       
    [PermissionState] = perm.[state_desc],       
    [ObjectType] = obj.type_desc,--perm.[class_desc],   
    [ObjectName] = OBJECT_NAME(perm.major_id),
    [ColumnName] = col.[name]
FROM        sys.database_role_members members --Role/member associations
JOIN        sys.database_principals roleprinc ON roleprinc.[principal_id] = members.[role_principal_id] --Roles
JOIN        sys.database_principals memberprinc ON memberprinc.[principal_id] = members.[member_principal_id] --Role members (database users)
LEFT JOIN        sys.login_token ulogin on memberprinc.[sid] = ulogin.[sid] --Login accounts
LEFT JOIN        sys.database_permissions perm ON perm.[grantee_principal_id] = roleprinc.[principal_id] --Permissions
LEFT JOIN        sys.columns col on col.[object_id] = perm.major_id AND col.[column_id] = perm.[minor_id] --Table columns 
LEFT JOIN        sys.objects obj ON perm.[major_id] = obj.[object_id]

UNION
SELECT  --List all access provisioned to the public role, which everyone gets by default
    [UserName] = '{All Users}',
    [UserType] = '{All Users}', 
    [DatabaseUserName] = '{All Users}',       
    [Role] = roleprinc.[name],      
    [PermissionType] = perm.[permission_name],       
    [PermissionState] = perm.[state_desc],       
    [ObjectType] = obj.type_desc,--perm.[class_desc],  
    [ObjectName] = OBJECT_NAME(perm.major_id),
    [ColumnName] = col.[name]
FROM sys.database_principals roleprinc --Roles
LEFT JOIN        sys.database_permissions perm ON perm.[grantee_principal_id] = roleprinc.[principal_id] --Role permissions
LEFT JOIN        sys.columns col on col.[object_id] = perm.major_id AND col.[column_id] = perm.[minor_id] --Table columns
JOIN sys.objects obj ON obj.[object_id] = perm.[major_id] --All objects   
WHERE
    roleprinc.[type] = 'R' AND --Only roles
    roleprinc.[name] = 'public' AND --Only public role
    obj.is_ms_shipped = 0 --Only objects of ours, not the MS objects
ORDER BY        --    princ.[Name],
    OBJECT_NAME(perm.major_id),
    col.[name],
    perm.[permission_name],
    perm.[state_desc],
    obj.type_desc--perm.[class_desc]



In [ ]:
SELECT *
  FROM MyTable
  WHERE [dateColumn] > '3/1/2009' AND [dateColumn] <= DATEADD(day,1,'3/31/2009') 
        --make it inclusive for a datetime type
    AND DATEPART(hh,[dateColumn]) >= 6 AND DATEPART(hh,[dateColumn]) <= 22 
        -- gets the hour of the day from the datetime
    AND DATEPART(dw,[dateColumn]) >= 3 AND DATEPART(dw,[dateColumn]) <= 5 
        -- gets the day of the week from the datetime

### 50 Important Queries In SQL Server
    https://www.c-sharpcorner.com/article/50-important-queries-in-sql-server/

### A importância do nível de compatibilidade do banco de dados no SQL Server

    https://www.sqlskills.com/blogs/glenn/database-compatibility-level-in-sql-server/

In [ ]:
-- Field in table

SELECT t.name AS table_name, SCHEMA_NAME(schema_id) AS schema_name, c.name AS column_name
FROM sys.tables AS t
INNER JOIN sys.columns c ON t.OBJECT_ID = c.OBJECT_ID
WHERE c.name LIKE '%Campo%'
ORDER BY schema_name, table_name;

In [ ]:
-- DROP - ALL - CONSTRAINT

CREATE TABLE #Commands (Command VARCHAR(MAX))

INSERT #Commands
SELECT 'ALTER TABLE ' + QUOTENAME(RC.CONSTRAINT_SCHEMA)
    + '.' + QUOTENAME(KCU1.TABLE_NAME)
    + ' DROP CONSTRAINT ' + QUOTENAME(rc.CONSTRAINT_NAME) + '; '
FROM INFORMATION_SCHEMA.REFERENTIAL_CONSTRAINTS AS RC

INNER JOIN INFORMATION_SCHEMA.KEY_COLUMN_USAGE AS KCU1
    ON KCU1.CONSTRAINT_CATALOG = RC.CONSTRAINT_CATALOG
    AND KCU1.CONSTRAINT_SCHEMA = RC.CONSTRAINT_SCHEMA
    AND KCU1.CONSTRAINT_NAME = RC.CONSTRAINT_NAME
WHERE ORDINAL_POSITION=1

--SELECT * FROM #Commands

DECLARE @Command VARCHAR(MAX)
DECLARE curCommand CURSOR FOR
SELECT Command FROM #Commands

OPEN curCommand

FETCH NEXT FROM curCommand INTO @Command

WHILE @@FETCH_STATUS =0
BEGIN

    EXEC(@Command)
    FETCH NEXT FROM curCommand INTO @Command

END

CLOSE curCommand
DEALLOCATE curCommand

DROP TABLE #Commands

https://www.tiagoneves.net/blog/procedures-nao-documentadas-sp_msforeachdb-e-sp_msforeachtable/

In [ ]:
-- TRUNCATE TABLE sp_MSForEachTable
EXEC sp_MSForEachTable "TRUNCATE TABLE ?"

In [ ]:
-- DISABLE ALL TRIGGER
sp_msforeachtable 'ALTER TABLE ? DISABLE TRIGGER all'

In [ ]:
-- DISABLE  TRIGGER SELETIVO

select 'alter table '+ (select Schema_name(schema_id) from sys.objects o where o.object_id = parent_id) + '.'+ object_name(parent_id) + ' DISABLE TRIGGER ' + Name as EnableScript
from sys.triggers t 
where is_disabled = 0 And Name not like '%_IR'

In [ ]:
-- Cross-database dependencies
SELECT  OBJECT_NAME (referencing_id) AS referencing_object, referenced_database_name, referenced_schema_name, referenced_entity_name
FROM sys.sql_expression_dependencies
WHERE referenced_database_name IS NOT NULL AND is_ambiguous = 0
order by OBJECT_NAME (referencing_id)

In [ ]:
-- Auto-increment -- Desativa - Insert - Reativar
SET IDENTITY_INSERT masterTbl ON  --Desabilita o IDENTITY

INSERT INTO masterTbl (id, name) VALUES (1, 'MNO') --Consegue inserir

SET IDENTITY_INSERT masterTbl OFF  --Habilita o IDENTITY

In [ ]:
-- Reseed Identity
DBCC CHECKIDENT(tblPerson, RESEED, 0)

In [ ]:
-- DROP - ALL - SCHEMA
USE [Processo]
GO

/****** Object:  Schema [TCE\81344546404]    Script Date: 16/06/2020 10:34:53 ******/
--DROP SCHEMA [TCE\81344546404]
--GO


select 'DROP SCHEMA ['+ s.name + ']' as schema_name, u.name as schema_owner
from sys.schemas s
inner join sys.sysusers u on u.uid = s.principal_id
--WHERE LEFT( s.name, 4) = 'TCE\' AND s.name NOT IN ('TCE\67154832487', 'TCE\08283413465', 'TCE\63729741420', 'TCE\usuários do dominio')
order by s.name




In [ ]:
-- Compare stored procedures across multiple databases (SQL Server)
    -- https://stackoverflow.com/questions/2133876/compare-stored-procedures-across-multiple-databases-sql-server

    -- https://www.mssqltips.com/sqlservertip/2626/verify-sql-server-stored-procedures-are-the-same-on-multiple-servers/

In [ ]:
GRANT Controle on Certificate

GRANT CONTROL ON CERTIFICATE :: CertificadoCg0101_v3 TO uCG0101Coleta;
GO
---
GRANT CONTROL ON SYMMETRIC KEY :: ChaveSimetricaCg0101 TO uCG0101;
GO

In [ ]:
-- Check who has access to symmetric keys

--GRANT CONTROL  para certificado e chave simetrica

-- grant CONTROL ON CERTIFICATE::InserirChave TO Inserirusuario
-- grant CONTROL ON SYMMETRIC KEY::InserirChave TO Inserirusuario

/*
SELECT * FROM SYS.symmetric_keys



select
      [database] = db_name()
    , u.name
    , p.permission_name
    , p.class
    , p.class_desc
    , ObjectNameForObjectORColumn
        = object_name(p.major_id) 
    , objectNameActual
        = case class_desc
            when 'SYMMETRIC_KEYS' then sm.name              
            when 'CERTIFICATE' then [cert].name             
      end
    , state_desc 
from sys.database_permissions  p 
inner join sys.database_principals	u		on p.grantee_principal_id = u.principal_id
left outer join sys.symmetric_keys	sm		on p.major_id = sm.symmetric_key_id and p.class_desc = 'SYMMETRIC_KEYS'
left outer join sys.certificates	[cert]	on p.major_id = [cert].[certificate_id] and p.class_desc = 'CERTIFICATE'
where class_desc in('SYMMETRIC_KEYS', 'CERTIFICATE')
order by u.name
*/

Check Fragmentation

Failed allocate pages: FAIL_PAGE_ALLOCATION 1

-- https://dba.stackexchange.com/questions/141236/failed-allocate-pages-fail-page-allocation-1

Linux Logging Basics

https://www.loggly.com/ultimate-guide/linux-logging-basics/